In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Define a custom Dataset
class EEGEMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Transformer model with real-time synchronization and cross-modality attention
class EEGEMGTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes, dropout_rate=0.5):
        super(EEGEMGTransformer, self).__init__()
        self.align_eeg = nn.Linear(input_dim, input_dim)
        self.align_emg = nn.Linear(input_dim, input_dim)
        
        self.eeg_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, batch_first=True, dropout=dropout_rate),
            num_layers=2)
        self.emg_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, batch_first=True, dropout=dropout_rate),
            num_layers=2)
        
        self.eeg_projector = nn.Linear(input_dim, hidden_dim)
        self.emg_projector = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout_rate)
        
        # Cross-modality attention with learnable weights
        self.cross_attention_weights = nn.Parameter(torch.rand(2, hidden_dim), requires_grad=True)
        self.cross_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, eeg, emg):
        eeg = self.align_eeg(eeg)
        emg = self.align_emg(emg)
        eeg_features = self.eeg_encoder(eeg)
        emg_features = self.emg_encoder(emg)
        
        eeg_features = self.eeg_projector(eeg_features)
        emg_features = self.emg_projector(emg_features)
        
        eeg_features = self.dropout(eeg_features)
        emg_features = self.dropout(emg_features)
        
        # Weighted combination of EEG and EMG
        combined_features = (
            self.cross_attention_weights[0] * eeg_features + self.cross_attention_weights[1] * emg_features
        )
        combined, _ = self.cross_attention(combined_features, combined_features, combined_features)
        
        output = self.fc(combined.mean(dim=1))
        return output

# Online adaptation function with regularization
def online_adaptation_with_regularization(
    model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001
):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)

    # Lower learning rate for stable online updates
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)
    
    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            eeg = X_batch[:, :input_dim].unsqueeze(1)
            emg = X_batch[:, input_dim:].unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(eeg, emg)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()

        # Evaluate on the validation set after each cycle
        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

# Train function with early stopping
def train_model_with_weight_decay(model, train_loader, val_loader, criterion, epochs=15, patience=5, lr=0.0005):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            eeg = X_batch[:, :input_dim].unsqueeze(1)
            emg = X_batch[:, input_dim:].unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(eeg, emg)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                eeg = X_batch[:, :input_dim].unsqueeze(1)
                emg = X_batch[:, input_dim:].unsqueeze(1)
                outputs = model(eeg, emg)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_state)
                break

# Evaluation function
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (
    balanced_accuracy_score,
    cohen_kappa_score,
    matthews_corrcoef,
    log_loss,
    roc_auc_score,
    average_precision_score
)
import numpy as np

# Updated evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []  # For probability outputs

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            eeg = X_batch[:, :input_dim].unsqueeze(1)
            emg = X_batch[:, input_dim:].unsqueeze(1)
            outputs = model(eeg, emg)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_scores.extend(probs.cpu().numpy())

    # Compute classification metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    log_loss_value = log_loss(y_true, y_scores)

    # AUROC and AUPRC (only for binary or multiclass with probability scores)
    try:
        auroc = roc_auc_score(y_true, y_scores, multi_class='ovr', average='weighted')
        auprc = average_precision_score(y_true, y_scores, average='weighted')
    except ValueError:
        auroc = None
        auprc = None

    # Mean Per-Class Error (MPCE)
    cm = confusion_matrix(y_true, y_pred)
    per_class_error = 1 - (np.diag(cm) / cm.sum(axis=1))
    mpce = np.mean(per_class_error)

    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}, Accuracy: {accuracy:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}, Cohen's Kappa: {kappa:.2f}, MCC: {mcc:.2f}")
    print(f"Log Loss: {log_loss_value:.4f}, "
          f"AUROC: {auroc:.2f}" if auroc is not None else "AUROC: N/A",
          f"AUPRC: {auprc:.2f}" if auprc is not None else "AUPRC: N/A")
    print(f"MPCE: {mpce:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'cohen_kappa': kappa,
        'mcc': mcc,
        'log_loss': log_loss_value,
        'auroc': auroc,
        'auprc': auprc,
        'mpce': mpce
    }


# k-Fold Cross-Validation with Online Adaptation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_results = []
online_adaptation_percentage = 0.3  # 10% data for online adaptation

for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"Fold {fold + 1}/{k}")
    X_train, X_val = combined_data.values[train_index], combined_data.values[val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    
    train_dataset = EEGEMGDataset(X_train, y_train)
    val_dataset = EEGEMGDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    input_dim = X_train.shape[1] // 2
    hidden_dim = 256
    num_heads = 4
    num_classes = len(np.unique(labels))
    
    model = EEGEMGTransformer(input_dim=input_dim, hidden_dim=hidden_dim, num_heads=num_heads, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()

    # Train model
    train_model_with_weight_decay(model, train_loader, val_loader, criterion)

    print("Before Online Adaptation:")
    metrics = evaluate_model(model, val_loader)  # Returns a dictionary
    fold_results.append(metrics)

    # Online adaptation
    online_data_size = int(len(X_val) * online_adaptation_percentage)
    online_X, online_y = X_val[:online_data_size], y_val[:online_data_size]
    print(f"Online Adaptation using {online_data_size} samples.")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.01)
    online_adaptation_with_regularization(
        model, optimizer, online_X, online_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001
    )

    print("After Online Adaptation:")
    metrics_after_adaptation = evaluate_model(model, val_loader)
    fold_results[-1].update(metrics_after_adaptation)


# **Place the best model saving code here**
best_model_index = np.argmax([result['accuracy'] for result in fold_results])
torch.save(model.state_dict(), 'EEGEMGTransformer_best.pth')
print(f"Best model (Fold {best_model_index + 1}) saved as EEGEMGTransformer_best.pth")

# Compute average metrics across k-folds
average_metrics = {metric: np.mean([result[metric] for result in fold_results]) for metric in fold_results[0]}
print(f"\nAverage Metrics across {k} folds:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.2f}")
